In [127]:
#take llm's to interpret abbrevations and expand the query accordingly for better result
#advanced: detect if term is medical and then instead of llm's use a dictionary as it is more precise
#optionals:
    #detect if query is a question query and prompt LLM to answer question, then expand query with keywords
    #get query and analyze most important part through LLM, then append that part multiple times to query     

# LLM's interprete abbrevations

In [128]:
#imports
import openai
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [129]:
client = openai.OpenAI() #connect to OpenAI API

In [130]:
def ask_gpt(prompt, model="gpt-4", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [131]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

In [132]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

alter query

In [133]:
#iterate through each query in pt_dataset.get_topics('query')
#for query decide whether this query contains an abbrevation or not
#if query contains abbrevation expand the query through writing out the abbrevation

In [134]:
querys = pt_dataset.get_topics('query')
querys.head(71)

,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm
3,4,stemming for arabic languages
4,5,audio based animal recognition
...,...,...
63,65,information in different language
64,66,abbreviations in queries
65,67,lemmatization algorithms
66,68,filter ad rich documents


In [135]:
print(querys['query'][0])

retrieval system improving effectiveness


determine if query contains abbrevation using gpt

In [136]:
answers = dict()
for i in range(len(querys)):
    determine_abbrevation = f""" 
    You are an scientific expert especially in the domain of Information Retrieval. Your task is to detect whether
    a given query, which is given as a text below delimited by triple quotes, contains an abbrevation then answer with yes or not then answer with no.
    For example given a query 'What is crx' you should answer yes, since cxr is the abbrevation for the medical term
    'chest X-Ray'. However if the given query is 'What is Information Retrieval' you should answer no, since there is no
    abbrevation in the query.

    query: '''{querys['query'][i]}'''
    """
    answer = ask_gpt(prompt=determine_abbrevation) #check answer more carefully perhaps model will return not only {yes,no}
    #print(answer)
    qid = str(querys['qid'][i])
    if "yes" in answer.lower().strip():
        answers[qid] = True
    else:
        answers[qid] = False
print(answers)

{'1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': True, '12': False, '13': False, '14': False, '15': False, '16': False, '17': False, '19': False, '20': False, '21': False, '22': False, '23': False, '24': False, '25': False, '26': False, '27': False, '28': False, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': False, '48': False, '49': False, '50': False, '51': False, '52': False, '53': False, '54': False, '55': True, '56': False, '57': False, '58': False, '59': True, '60': False, '61': False, '62': False, '63': False, '64': False, '65': False, '66': False, '67': False, '68': False, '18': False}


Expand query which contains abbrevation

In [137]:
#could be more efficient if in answers text is saved only for qid where abbrevation=yes
for key in answers.keys():
    if bool(answers[key]):
        #find query
        for i in range(len(querys)):
            if querys['qid'][i] == key:
                query= querys['query'][i]
                print("Old query: ",query)
        #ask gpt to expand query
        expand = f""" 
        You are an scientific expert especially in the domain of Information Retrieval. Your are given a query, which is below
        delimited by triple quotes, which contains an abbrevation. Your task is to identify the abbrevation and write it, then
        concat the written out abbrevation with the original query and return this new query as string only.
        For example given a query 'What is crx' you should detect that the abbrevation is crx, since cxr is the abbrevation for the medical term
        'chest X-Ray', then your should concat the abbrevation 'chest X-Ray' with the originial query resulting in a new query 'chest X-Ray What is crx' which
        you should return. Another example, given the query 'Algorithms of nlp' you should detect that the abbrevation is nlp, since nlp is the abbrevation
        for the term 'natural language processing', then you should concat the abbrevation 'natural language processing' with the original query 'Algorithms of nlp' 
        resulting in a new query 'natural language processing Algorithms of nlp' which you should return.
        Please only answer with the new query.

        query: '''{query}'''
        """
        new_query = ask_gpt(prompt=expand).lower().strip().replace("'", " ").replace('"', ' ')
        print("New query: ",new_query)
        #overwrite old query
        querys['query'][i] = new_query

Old query:  algorithm acceleration with nvidia cuda
New query:   compute unified device architecture algorithm acceleration with nvidia cuda 
Old query:  bm25
New query:  okapi bm25 bm25
Old query:  what is ahp
New query:   analytic hierarchy process what is ahp 


In [138]:
print(querys['query'])

0     retrieval system improving effectiveness
1     machine learning language identification
2                social media detect self harm
3                stemming for arabic languages
4               audio based animal recognition
                        ...                   
63           information in different language
64                    abbreviations in queries
65                    lemmatization algorithms
66                    filter ad rich documents
67     analytic hierarchy process what is ahp 
Name: query, Length: 68, dtype: object


In [139]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [140]:
print('Now we do the retrieval...')
run = bm25(querys)

print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...


Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness
5,1,82472,1998.sigirconf_conference-98.15,5,13.901647,retrieval system improving effectiveness
6,1,94415,2008.cikm_conference-2008.183,6,13.808208,retrieval system improving effectiveness
7,1,17496,O01-2005,7,13.749449,retrieval system improving effectiveness
8,1,82490,1998.sigirconf_conference-98.33,8,13.735541,retrieval system improving effectiveness
9,1,124801,2006.ipm_journal-ir0volumeA42A3.2,9,13.569263,retrieval system improving effectiveness


In [141]:
run[65283:66283] #for query what is ahp the retrieval systems now finds 1000 docs, as before it found 3: illustrated below

,qid,docid,docno,rank,score,query
65283,18,124882,2006.ipm_journal-ir0volumeA42A4.0,0,27.781898,analytic hierarchy process what is ahp
65284,18,114774,2017.ijirr_journal-ir0volumeA7A3.1,1,25.656730,analytic hierarchy process what is ahp
65285,18,118484,2013.jasis_journal-ir0volumeA64A7.12,2,22.011661,analytic hierarchy process what is ahp
65286,18,95978,2007.cikm_conference-2007.51,3,19.821424,analytic hierarchy process what is ahp
65287,18,114140,2018.wwwjournals_journal-ir0volumeA21A1.3,4,18.963078,analytic hierarchy process what is ahp
...,...,...,...,...,...,...
66278,18,38713,D17-1123,995,7.263059,analytic hierarchy process what is ahp
66279,18,58604,C90-3076,996,7.263059,analytic hierarchy process what is ahp
66280,18,81248,2004.sigirconf_conference-2004.62,997,7.263059,analytic hierarchy process what is ahp
66281,18,82476,1998.sigirconf_conference-98.19,998,7.263059,analytic hierarchy process what is ahp


bm25:

In [145]:
run2 = bm25(pt_dataset.get_topics('text'))

In [146]:
run2[56279:56284] #query only finds 3 docs

,qid,docid,docno,rank,score,query
56279,58,113091,2005.ir_journal-ir0volumeA8A4.1,999,13.112250,architecture of web search engine
56280,59,124882,2006.ipm_journal-ir0volumeA42A4.0,0,10.066961,what is ahp
56281,59,114140,2018.wwwjournals_journal-ir0volumeA21A1.3,1,8.997160,what is ahp
56282,59,126037,2016.ipm_journal-ir0volumeA52A6.8,2,6.783162,what is ahp
56283,60,84131,2003.sigirconf_conference-2003.73,0,6.025493,what is information retrieval


runs

In [144]:
persist_and_normalize_run(run, system_name='llm-expansion', default_output='../runs') #new llm expansion
#persist_and_normalize_run(run2, system_name='bm25-baseline', default_output='../runs') #bm25 unmodified

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
